<a href="https://www.kaggle.com/code/tipatle/utkfacesimplemodel?scriptVersionId=126273845" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.rRead_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/utkface-new/UTKFace/26_0_2_20170104023102422.jpg.chip.jpg
/kaggle/input/utkface-new/utkface_aligned_cropped/UTKFace/26_0_2_20170104023102422.jpg.chip.jpg
/kaggle/input/utkface-new/utkface_aligned_cropped/crop_part1/26_0_2_20170104023102422.jpg.chip.jpg
/kaggle/input/utkface-new/crop_part1/26_0_2_20170104023102422.jpg.chip.jpg


In [14]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import os
from keras.preprocessing.image import ImageDataGenerator

In [15]:
folder_path="/kaggle/input/utkface-new/UTKFace"
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [16]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [17]:
df.head()

,age,gender,img
0,26,0,26_0_2_20170104023102422.jpg.chip.jpg
1,22,1,22_1_1_20170112233644761.jpg.chip.jpg
2,21,1,21_1_3_20170105003215901.jpg.chip.jpg
3,28,0,28_0_0_20170117180555824.jpg.chip.jpg
4,17,1,17_1_4_20170103222931966.jpg.chip.jpg


In [18]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [19]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(256,256),
                                                    batch_size = 32,
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    batch_size = 32,
                                                    target_size=(256,256),
                                                  class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [20]:
from keras.layers import *
from keras.models import Model
import keras
import keras_tuner

In [21]:
def build_model(hp):
    layers = hp.Int(f"layers", min_value=2, max_value=6, step=1)
    input = keras.Input(shape=(256, 256, 3))
    initializer = hp.Choice('initializer', ['glorot_uniform', 'glorot_normal'])
    model = input
    for i in range(layers):
        if(i!=0):
            if(initializer=='glorot_uniform'):
                model = Conv2D(hp.Int(f"units{i+1}", min_value=4, max_value=512, step=8),
                           (3, 3), 
                           padding='same', 
                           activation='relu', 
                           kernel_initializer=keras.initializers.GlorotUniform(seed=42))(model)
            elif(initializer=='glorot_normal'):
                model = Conv2D(hp.Int(f"units{i+1}", min_value=4, max_value=512, step=8),
                           (3, 3), 
                           padding='same', 
                           activation='relu', 
                           kernel_initializer=keras.initializers.GlorotNormal(seed=42))(model)
        else:
            if(initializer=='glorot_uniform'):
                model = Conv2D(hp.Int(f"units{i+1}", min_value=4, max_value=512, step=8),
                           (3, 3), 
                           padding='same', 
                           activation='relu', 
                           kernel_initializer=keras.initializers.GlorotUniform(seed=42))(input)
            elif(initializer=='glorot_normal'):
                model = Conv2D(hp.Int(f"units{i+1}", min_value=4, max_value=512, step=8),
                           (3, 3), 
                           padding='same', 
                           activation='relu', 
                           kernel_initializer=keras.initializers.GlorotNormal(seed=42))(input)
            
        model = MaxPool2D((2, 2), 
                          strides=hp.Int(f'stride{i+1}', min_value=1, max_value=2))(model)
        model = Dropout(
            rate=hp.Choice(f'rate{i+1}', [0.0, 0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5 ]),seed=42)(model)
        
    model = Flatten()(model)
    output1 = hp.Int(f"output1", min_value=1, max_value=10, step=1)
    output2 = hp.Int(f"output2", min_value=1, max_value=10, step=1)
    for i in range(output1):
        if(i!=0):
            dense1 = Dense(hp.Int(f"d1units{i+1}", min_value=1, max_value=512, step=8),
                           activation='relu')(dense1)
        else:
            dense1 = Dense(hp.Int(f"d1units{i+1}", min_value=1, max_value=512, step=8),
                           activation='relu')(model)
        dense1 = Dropout(
            rate=hp.Choice(f'd1rate{i+1}', [0.0, 0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5 ]), seed=42)(dense1)
        
    for i in range(output2):
        if(i!=0):
            dense2 = Dense(hp.Int(f"d2units{i+1}", min_value=1, max_value=512, step=8),
                           activation='relu')(dense2)
        else:
            dense2 = Dense(hp.Int(f"d2units{i+1}", min_value=1, max_value=512, step=8),
                           activation='relu')(model)
        dense2 = Dropout(
            rate=hp.Choice(f'd2rate{i+1}', [0.0, 0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5 ]), seed=42)(dense2)
    
    o1 = Dense(1, activation='linear')(dense1)
    o2 = Dense(1, activation='sigmoid')(dense2)
    Model = keras.Model(inputs=input, outputs=[o1, o2], name="UTKFaceModel")
    losses = [[tf.keras.losses.MeanSquaredError()], [tf.keras.losses.BinaryCrossentropy()]]
    metrics = [[tf.keras.metrics.MeanAbsoluteError()], [tf.keras.metrics.BinaryAccuracy()]]
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2)                  
    optimizer = hp.Choice('optimizer', ['adam', 'rmsprop'])
    if optimizer=='adam':
        Model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=lr),
            loss=losses,
            loss_weights=[4.0, 1.0],
            metrics=metrics
        )
    elif optimizer=='rmsprop':
        Model.compile(
            optimizer=keras.optimizers.RMSprop(learning_rate=lr),
            loss=losses,
            loss_weights=[1.0, 1.0],
            metrics=metrics
        )
    return Model

In [22]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective=[['val_loss'], ['val_accuracy']],
    max_trials=5,
    overwrite=True,
    project_name="UTKFACE",
)

In [23]:
tuner.search( train_generator, epochs=5, validation_data=test_generator)

Trial 1 Complete [00h 38m 26s]
multi_objective: 0.0

Best multi_objective So Far: 0.0
Total elapsed time: 00h 38m 26s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
3                 |3                 |layers
glorot_uniform    |glorot_uniform    |initializer
212               |260               |units1
1                 |2                 |stride1
0.2               |0.2               |rate1
452               |500               |units2
2                 |2                 |stride2
0.2               |0.4               |rate2
2                 |5                 |output1
3                 |2                 |output2
433               |177               |d1units1
0.35              |0.3               |d1rate1
185               |177               |d2units1
0.2               |0.4               |d2rate1
0.0092397         |0.00041434        |lr
adam              |rmsprop           |optimizer
476               |4                 |units3
1                 |1     

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/keras_tuner/engine/tuner.py", line 158, in _try_build
    model = self._build_hypermodel(hp)
  File "/opt/conda/lib/python3.7/site-packages/keras_tuner/engine/tuner.py", line 146, in _build_hypermodel
    model = self.hypermodel.build(hp)
  File "/tmp/ipykernel_23/2910297278.py", line 48, in build_model
    activation='relu')(model)
  File "/opt/conda/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/opt/conda/lib/python3.7/site-packages/keras/backend.py", line 2105, in random_uniform
    seed=seed,
tensorflow.python.framework.errors_impl.ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[7556976,433] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Statel

Invalid model 0/5


Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/keras_tuner/engine/tuner.py", line 158, in _try_build
    model = self._build_hypermodel(hp)
  File "/opt/conda/lib/python3.7/site-packages/keras_tuner/engine/tuner.py", line 146, in _build_hypermodel
    model = self.hypermodel.build(hp)
  File "/tmp/ipykernel_23/2910297278.py", line 48, in build_model
    activation='relu')(model)
  File "/opt/conda/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/opt/conda/lib/python3.7/site-packages/keras/backend.py", line 2105, in random_uniform
    seed=seed,
tensorflow.python.framework.errors_impl.ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[7556976,433] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Statel

Invalid model 1/5


...

Invalid model 3/5


Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/keras_tuner/engine/tuner.py", line 158, in _try_build
    model = self._build_hypermodel(hp)
  File "/opt/conda/lib/python3.7/site-packages/keras_tuner/engine/tuner.py", line 146, in _build_hypermodel
    model = self.hypermodel.build(hp)
  File "/tmp/ipykernel_23/2910297278.py", line 48, in build_model
    activation='relu')(model)
  File "/opt/conda/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/opt/conda/lib/python3.7/site-packages/keras/backend.py", line 2105, in random_uniform
    seed=seed,
tensorflow.python.framework.errors_impl.ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[7556976,433] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Statel

Invalid model 4/5
Invalid model 5/5


Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/keras_tuner/engine/tuner.py", line 158, in _try_build
    model = self._build_hypermodel(hp)
  File "/opt/conda/lib/python3.7/site-packages/keras_tuner/engine/tuner.py", line 146, in _build_hypermodel
    model = self.hypermodel.build(hp)
  File "/tmp/ipykernel_23/2910297278.py", line 48, in build_model
    activation='relu')(model)
  File "/opt/conda/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/opt/conda/lib/python3.7/site-packages/keras/backend.py", line 2105, in random_uniform
    seed=seed,
tensorflow.python.framework.errors_impl.ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[7556976,433] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Statel

RuntimeError: Too many failed attempts to build model.

In [ ]:
Model = tuner.get_best_models(num_models=1)[0]
tf.keras.utils.plot_model(
    model = Model,
    show_shapes=True,
)

In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping( monitor = 'val_loss', patience=3),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5', save_freq="epoch",),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]
history = Model.fit(
    train_generator, 
    epochs = 50, 
    validation_data=test_generator, 
    batch_size=32, 
    validation_batch_size=32,
    use_multiprocessing=True, 
    callbacks=my_callbacks)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
plt.title("Loss vs Val_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot(history.history['loss'], color='red')
plt.plot(history.history['val_loss'], color='green')
plt.xticks(range(0, len(history.history['loss'])+1, 1))
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
plt.title("Output1_loss vs Output1_val_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot(history.history['dense_15_loss'], color='red')
plt.plot(history.history['val_dense_15_loss'], color='green')
plt.xticks(range(0, len(history.history['loss'])+1, 1))
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
plt.title("Output2_loss vs Output2_val_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot(history.history['dense_14_loss'], color='red')
plt.plot(history.history['val_dense_14_loss'], color='green')
plt.xticks(range(0, len(history.history['loss'])+1, 1))
plt.show()

In [ ]:
!mkdir -p saved_model
Model.save('saved_model/my_model')
Model.save('my_model.h5')